
Spark 유료버전에는 기본으로 Unity Catalog만 설치되며 Community 버전이나 다른 오픈소스에서는 있는 환경 변수들이 존재하지 않음
또한 보안 때문에 로컬 폴더와 DBFS간의 데이터를 주고 받는 것이 금지됨. 이번에는 orders.csv 파일을 S3에서 바로 다운로드받을 예정

In [0]:
spark.conf.get("spark.sql.catalogImplementation", "Not Set")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1314523540085783>, line 1
----> 1 spark.conf.get("spark.sql.catalogImplementation", "Not Set")

File /databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/conf.py:66, in RuntimeConf.get(self, key, default)
     62     op_get_with_default = proto.ConfigRequest.GetWithDefault(
     63         pairs=[proto.KeyValue(key=key, value=cast(Optional[str], default))]
     64     )
     65     operation = proto.ConfigRequest.Operation(get_with_default=op_get_with_default)
---> 66 result = self._client.config(operation)
     67 return result.pairs[0][1]

File /databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/client/core.py:1726, in SparkConnectClient.config(self, operation)
   1724     raise SparkConnectException("Invalid state during retry exception handling.")
   1725 except Exception as error:
-> 172

In [0]:
spark.conf.get("spark.sql.warehouse.dir")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1314523540085785>, line 1
----> 1 spark.conf.get("spark.sql.warehouse.dir")

File /databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/conf.py:66, in RuntimeConf.get(self, key, default)
     62     op_get_with_default = proto.ConfigRequest.GetWithDefault(
     63         pairs=[proto.KeyValue(key=key, value=cast(Optional[str], default))]
     64     )
     65     operation = proto.ConfigRequest.Operation(get_with_default=op_get_with_default)
---> 66 result = self._client.config(operation)
     67 return result.pairs[0][1]

File /databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/client/core.py:1726, in SparkConnectClient.config(self, operation)
   1724     raise SparkConnectException("Invalid state during retry exception handling.")
   1725 except Exception as error:
-> 1726     self._handle_

In [0]:
spark.sql("SHOW CATALOGS").show()

+---------+
|  catalog|
+---------+
|  samples|
|   system|
|workspace|
+---------+



In [0]:
spark.sql("SHOW DATABASES").show()

+------------------+
|      databaseName|
+------------------+
|           default|
|               dev|
|information_schema|
+------------------+



In [0]:
spark.sql("SHOW TABLES").show()

+--------+------------+-----------+
|database|   tableName|isTemporary|
+--------+------------+-----------+
|     dev|      orders|      false|
|     dev|orders_count|      false|
|        |      _sqldf|       true|
+--------+------------+-----------+



In [0]:
spark.sql("SELECT CURRENT_CATALOG()").show()

+-----------------+
|CURRENT_CATALOG()|
+-----------------+
|        workspace|
+-----------------+



In [0]:
spark.sql("SELECT CURRENT_DATABASE()").show()

+------------------+
|CURRENT_DATABASE()|
+------------------+
|               dev|
+------------------+



In [0]:
# 새로 데이터베이스를 하나 만들고 그걸 사용
spark.sql("CREATE DATABASE IF NOT EXISTS DEV")
spark.sql("USE DEV")

DataFrame[]

In [0]:
spark.sql("SHOW DATABASES").show()

+------------------+
|      databaseName|
+------------------+
|           default|
|               dev|
|information_schema|
+------------------+



In [0]:
spark.sql("SELECT CURRENT_DATABASE()").show()

+------------------+
|CURRENT_DATABASE()|
+------------------+
|               dev|
+------------------+



In [0]:
# S3에서 바로 로딩. 읽기에는 권한이 필요없게 S3 bucket이 설정되어 있기에 가능
df = spark.read.csv(
    "s3a://s3-geospatial/orders.csv",
    inferSchema=True,
    header=True,
    sep ='\t'
)

In [0]:
df.show()

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
|862975819988|[{"name": "DAILY ...|
|862985191636|[{"name": "DAILY ...|
|870939295956|[{"name": "DAILY ...|
|880188063956|[{"name": "DAILY ...|
|933014601940|[{"name": "DAILY ...|
|934065930452|[{"name": "DAILY ...|
|938210722004|[{"name": "DAILY ...|
|944748331220|[{"name": "DAILY ...|
|862843896020|[{"name": "DAILY ...|
|862959763668|[{"name": "DAILY ...|
|870966558932|[{"name": "DAILY ...|
|887936647380|[{"name": "DAILY ...|
|908426477780|[{"name": "DAILY ...|
|921300107476|[{"name": "DAILY ...|
|932229710036|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 20 rows


In [0]:
df.write.saveAsTable("DEV.orders", mode="overwrite")

In [0]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|     dev|   orders|      false|
|        |   _sqldf|       true|
+--------+---------+-----------+



In [0]:
spark.catalog.listTables()

[Table(name='orders', catalog='workspace', namespace=['dev'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='_sqldf', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
%fs ls dbfs:/

path,name,size,modificationTime
dbfs:/Volumes/,Volumes/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0


In [0]:
spark.table("DEV.orders").show(10)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
|862975819988|[{"name": "DAILY ...|
|862985191636|[{"name": "DAILY ...|
|870939295956|[{"name": "DAILY ...|
|880188063956|[{"name": "DAILY ...|
|933014601940|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 10 rows


In [0]:
spark.sql("SELECT * FROM DEV.orders LIMIT 10").show()

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
|862975819988|[{"name": "DAILY ...|
|862985191636|[{"name": "DAILY ...|
|870939295956|[{"name": "DAILY ...|
|880188063956|[{"name": "DAILY ...|
|933014601940|[{"name": "DAILY ...|
+------------+--------------------+



In [0]:
%sql

SELECT * FROM DEV.orders LIMIT 10

order_id,items
860196503764,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1883727790094}]"
860292645076,"[{""name"": ""DAILY SPF \u2014 Bundle Set"", ""quantity"": 1, ""id"": 1883875377166}]"
860320956628,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1883919974414}]"
860321513684,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1883920793614}]"
862930665684,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1887913672718}]"
862975819988,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1887985827854}]"
862985191636,"[{""name"": ""DAILY SPF \u2014 Bundle Set"", ""quantity"": 1, ""id"": 1887999164430}]"
870939295956,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1900142264334}]"
880188063956,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1914170572814}]"
933014601940,"[{""name"": ""DAILY SPF"", ""quantity"": 1, ""id"": 1995572117518}]"


In [0]:
spark.sql("""
    CREATE TABLE DEV.orders_count AS
    SELECT order_id, COUNT(1) as count
    FROM DEV.orders
    GROUP BY 1""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("SHOW TABLES").show()

+--------+------------+-----------+
|database|   tableName|isTemporary|
+--------+------------+-----------+
|     dev|      orders|      false|
|     dev|orders_count|      false|
|        |      _sqldf|       true|
+--------+------------+-----------+



In [0]:
# 결과에서 tableType을 살펴볼 것!
spark.catalog.listTables()

[Table(name='orders', catalog='workspace', namespace=['dev'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='orders_count', catalog='workspace', namespace=['dev'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='_sqldf', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
%sql

SELECT *
FROM dev.orders_count

order_id,count
1296324231380,1
1256869527764,1
2209629307092,1
933014601940,1
1221504565460,1
2108798238932,1
2102309093588,1
1299495092436,1
2047024529620,1
2207727878356,1


External Table 예는 나중 "실습과 확장" 섹션에서 AWS EMR을 사용할 때 S3와 연동하는 걸로 대신 할 예정